In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import numpy as np

In [4]:
import os
from torchvision import datasets, transforms
import torch

train_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/val"

data_transform = {
    "train": transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
    "val": transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
}


image_datasets = {
    "train": datasets.ImageFolder(root=train_dir, transform=data_transform["train"]),
    "val": datasets.ImageFolder(root=val_dir, transform=data_transform["val"])
}

dataloader = {
    "train": torch.utils.data.DataLoader(dataset=image_datasets["train"], batch_size=16, shuffle=True),
    "val": torch.utils.data.DataLoader(dataset=image_datasets["val"], batch_size=16, shuffle=False)
}


In [5]:
class DatasetLoader:
    def __init__(self, train_dir, val_dir, batch_size=16):
        data_transform = {
            "train": transforms.Compose([
                transforms.Resize([224, 224]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ]),
            "val": transforms.Compose([
                transforms.Resize([224, 224]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])
        }

        self.image_datasets = {
            "train": datasets.ImageFolder(root=train_dir, transform=data_transform["train"]),
            "val": datasets.ImageFolder(root=val_dir, transform=data_transform["val"])
        }

        self.dataloaders = {
            "train": DataLoader(self.image_datasets["train"], batch_size=batch_size, shuffle=True),
            "val": DataLoader(self.image_datasets["val"], batch_size=batch_size, shuffle=False)
        }

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models

import torch.optim as optim


class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=True, init_weights=False):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(1024, num_classes)
        if init_weights:
            self._init_weight()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, start_dim=1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)

        if self.aux_logits and self.training:
            return x, aux2, aux1
        return x

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, dim=1)


class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.averagePool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)

        self.aux_classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 1024),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.averagePool(x)
        x = self.conv(x)
        x = torch.flatten(x, start_dim=1)
        x = self.aux_classifier(x)
        return x


class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x



In [ ]:
def train_and_evaluate(model, dataset_loader, num_epochs=10, learning_rate=0.001, patience=3, device='cuda'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = np.inf
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        # Training Phase
        model.train()
        train_loss, train_corrects, train_preds, train_labels = 0.0, 0, [], []

        for inputs, labels in tqdm(dataset_loader.dataloaders['train']):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # 前向传播
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # 只用主分支输出进行损失计算

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 更新损失和准确率
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_corrects += torch.sum(preds == labels.data)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss = train_loss / len(dataset_loader.image_datasets['train'])
        train_acc = train_corrects.double() / len(dataset_loader.image_datasets['train'])
        train_precision = precision_score(train_labels, train_preds, average='weighted')
        train_recall = recall_score(train_labels, train_preds, average='weighted')
        train_f1 = f1_score(train_labels, train_preds, average='weighted')
        train_auc = roc_auc_score(train_labels, train_preds, multi_class='ovr')

        print(f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} Precision: {train_precision:.4f} Recall: {train_recall:.4f} F1: {train_f1:.4f} AUC-ROC: {train_auc:.4f}")

        # Validation Phase
        model.eval()
        val_loss, val_corrects, val_preds, val_labels = 0.0, 0, [], []

        with torch.no_grad():
            for inputs, labels in dataset_loader.dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]

                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_loss = val_loss / len(dataset_loader.image_datasets['val'])
        val_acc = val_corrects.double() / len(dataset_loader.image_datasets['val'])
        val_precision = precision_score(val_labels, val_preds, average='weighted')
        val_recall = recall_score(val_labels, val_preds, average='weighted')
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        val_auc = roc_auc_score(val_labels, val_preds, multi_class='ovr')

        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} Precision: {val_precision:.4f} Recall: {val_recall:.4f} F1: {val_f1:.4f} AUC-ROC: {val_auc:.4f}")

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # 重置 patience 计数器
            best_model_wts = model.state_dict()  # 保存最佳模型权重
        else:
            patience_counter += 1
            print(f"Early Stopping Counter: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_wts)  # 加载最佳模型
                break

# 使用 DatasetLoader 类和训练模型
train_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/EE6483Project/datasets/datasets/val"
dataset_loader = DatasetLoader(train_dir, val_dir)

# 初始化并训练模型
model = GoogLeNet(num_classes=2, aux_logits=True)  # 假设是二分类问题
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_and_evaluate(model, dataset_loader, num_epochs=10, learning_rate=0.001, patience=3, device=device)

Epoch 1/10
--------------------


100%|██████████| 1250/1250 [11:24<00:00,  1.83it/s]


Train Loss: 0.6937 Acc: 0.4968 Precision: 0.4968 Recall: 0.4968 F1: 0.4968 AUC-ROC: 0.4968


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Val Loss: 0.6931 Acc: 0.5000 Precision: 0.2500 Recall: 0.5000 F1: 0.3333 AUC-ROC: 0.5000
Epoch 2/10
--------------------


100%|██████████| 1250/1250 [08:46<00:00,  2.38it/s]


Train Loss: 0.6933 Acc: 0.4982 Precision: 0.4982 Recall: 0.4982 F1: 0.4982 AUC-ROC: 0.4982


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Val Loss: 0.6932 Acc: 0.5000 Precision: 0.2500 Recall: 0.5000 F1: 0.3333 AUC-ROC: 0.5000
Early Stopping Counter: 1/3
Epoch 3/10
--------------------


 99%|█████████▉| 1236/1250 [08:42<00:06,  2.33it/s]